# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'), False)  # Load sampling strategies
models.search_threshold = -1


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/bidding_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/binfo_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-999000
INFO:tensorflow:Re

### Bidding

In [ ]:
vuln_ns, vuln_ew = False, True
hand = 'A.8xx.Jxx.AQxxxx'

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

auction = ["1S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, dds,  False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?


[{'call': '2C', 'insta_score': 0.559}]

In [ ]:
auction = ["1S","PASS","1N","PASS","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, dds, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']


[{'call': 'PASS', 'insta_score': 0.67}]

In [4]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


West always has the same cards because it's your hand and it's known to you

In [5]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Good quality: True
QJx.9xx.KT8xx.JT K9xx.KJT.AQ9x.K8 A.8xx.Jxx.AQxxxx T8xxx.AQxx.x.9xx 1.00
98x.JTxx.Q8xx.T8 KTxx.KQ9.AKx.J9x A.8xx.Jxx.AQxxxx QJxxx.Axx.T9x.Kx 1.00
K8xxx.Kxx.T8.T8x QJT.AJT9x.Q9x.Kx A.8xx.Jxx.AQxxxx 9xxx.Qx.AKxxx.J9 0.75
JT9xx.AQ.Q8xx.T9 KQxx.Tx.AT9xx.K8 A.8xx.Jxx.AQxxxx 8xx.KJ9xxx.K.Jxx 0.75
98x.Jx.A98xx.T8x KQx.AKQ.KTxx.KJ9 A.8xx.Jxx.AQxxxx JTxxxx.T9xxx.Q.x 0.75
T9x.QJ9x.xx.J98x QJ8x.Axxx.AT8x.K A.8xx.Jxx.AQxxxx Kxxxx.KT.KQ9x.Tx 0.75
Q98xx.J9x.9x.KJx KTxx.AKxx.Axx.9x A.8xx.Jxx.AQxxxx Jxx.QTx.KQT8x.T8 0.75
T9xxx.AKTx.T9xx. Qxx.xx.AKx.KT9xx A.8xx.Jxx.AQxxxx KJ8x.QJ9x.Q8x.J8 0.75
Q9xx.9xx.A9x.Jxx KJxxx.AJx.K8x.K8 A.8xx.Jxx.AQxxxx T8x.KQTx.QTxx.T9 1.00
Qxx.xx.Q9xxx.J8x JT.AJT9.AKTx.KTx A.8xx.Jxx.AQxxxx K98xxxx.KQxx.8.9 1.00
